In [5]:
!pip install fairlearn xgboost shap

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
astropy 5.3.4 requires numpy<2,>=1.21, but you have numpy 2.2.6 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.2.6 which is incompatible.
matplotlib 3.8.0 requires numpy<2,>=1.21, but you have numpy 2.2.6 which is incompatible.
pywavelets 1.5.0 requires numpy<2.0,>=1.22.4, but you have numpy 2.2.6 which is incompatible.


  Using cached fairlearn-0.13.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached slicer-0.0.8-py3-none-any.whl.metadata (4.0 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.8 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 648.8 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of pandas to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     --------------------------------- ------ 51.2/60.8 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 60.8/60.8 kB 1.1 MB/s eta 0:00:00
Using cached fairlearn-0.13.0-py3-none-any.whl (251 kB)
   ---------------------------------------- 0.0/548.0 kB ? eta -:

In [7]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from fairlearn.metrics import MetricFrame, selection_rate
from sklearn.preprocessing import LabelEncoder

# --- 1. LOAD THE DATA (Real UCI German Credit Data) ---
# We fetch directly from the UCI archive
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
columns = ['status', 'duration', 'credit_history', 'purpose', 'credit_amount', 
           'savings', 'employment', 'installment_rate', 'personal_status_sex', 
           'guarantors', 'residence_since', 'property', 'age', 'other_installments', 
           'housing', 'existing_credits', 'job', 'people_liable', 'telephone', 
           'foreign_worker', 'target']

# Read data (sep=' ' because it's an old school file)
df = pd.read_csv(url, sep=' ', header=None, names=columns)

# --- 2. PREPROCESSING & FEATURE ENGINEERING ---
# The target: 1 = Good, 2 = Bad. Let's make it 1 = Good, 0 = Bad
df['target'] = df['target'].map({1: 1, 2: 0})

# CRITICAL STEP: Extract the "Sensitive Feature" for Fairness
# In this dataset, 'personal_status_sex' is combined (e.g., 'A91' = male : divorced/separated)
# We need to isolate just 'Sex' or 'Age' to test for bias. 
# Let's create a binary 'sex' column (0=Female, 1=Male) based on common mappings for this dataset
# Mappings: A91, A93, A94 = Male; A92, A95 = Female
male_codes = ['A91', 'A93', 'A94']
df['sex_code'] = df['personal_status_sex'].apply(lambda x: 1 if x in male_codes else 0)

# Drop the original complex column to avoid leakage
X = df.drop(['target', 'personal_status_sex'], axis=1)
y = df['target']
sensitive_feature = df['sex_code'] # This is what we check fairness against

# Encode categorical variables for XGBoost
for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Split data
X_train, X_test, y_train, y_test, A_train, A_test = train_test_split(
    X, y, sensitive_feature, test_size=0.2, random_state=42
)

# --- 3. TRAIN THE "BIASED" MODEL (XGBoost) ---
# Note: We are NOT telling the model to be fair yet. We just want max accuracy
model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# --- 4. THE BIAS AUDIT (The "Money" Step) ---
# We use Fairlearn to compare how the model treats Men (1) vs Women (0)
metric_frame = MetricFrame(
    metrics=selection_rate,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=A_test
)

print("--- INITIAL MODEL AUDIT ---")
print(f"Overall Accuracy: {accuracy_score(y_test, y_pred):.2%}")
print(f"Selection Rate (Overall approval %): {selection_rate(y_test, y_pred):.2%}")
print("\n--- FAIRNESS CHECK ---")
print(f"Approval Rate for Women (Group 0): {metric_frame.by_group[0]:.2%}")
print(f"Approval Rate for Men   (Group 1): {metric_frame.by_group[1]:.2%}")
print(f"Difference: {metric_frame.difference():.2%}")

if metric_frame.difference() > 0.10:
    print("\nALERT: Significant Disparity Detected! (Gap > 10%)")
else:
    print("\nStatus: Relatively Fair")

--- INITIAL MODEL AUDIT ---
Overall Accuracy: 79.00%
Selection Rate (Overall approval %): 75.50%

--- FAIRNESS CHECK ---
Approval Rate for Women (Group 0): 73.21%
Approval Rate for Men   (Group 1): 76.39%
Difference: 3.17%

Status: Relatively Fair


In [15]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from fairlearn.metrics import MetricFrame, selection_rate
from sklearn.preprocessing import LabelEncoder

# --- 1. LOAD THE DATA (Real UCI German Credit Data) ---
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
columns = ['status', 'duration', 'credit_history', 'purpose', 'credit_amount', 
           'savings', 'employment', 'installment_rate', 'personal_status_sex', 
           'guarantors', 'residence_since', 'property', 'age', 'other_installments', 
           'housing', 'existing_credits', 'job', 'people_liable', 'telephone', 
           'foreign_worker', 'target']

# Read data
df = pd.read_csv(url, sep=' ', header=None, names=columns)

# --- 2. PREPROCESSING & FEATURE ENGINEERING ---
# The target: 1 = Good, 2 = Bad. Let's make it 1 = Good, 0 = Bad
df['target'] = df['target'].map({1: 1, 2: 0})

# CRITICAL CHANGE: SWITCH TO AGE BIAS
# We define "Unprivileged" (0) as Age <= 25, and "Privileged" (1) as Age > 25.
# This assumes the model might unfairly penalize young people due to lack of history.
df['age_group'] = df['age'].apply(lambda x: 1 if x > 25 else 0)

# Drop columns to avoid leakage, but KEEP 'age' in X so the model can actually learn the bias
X = df.drop(['target', 'personal_status_sex'], axis=1)
y = df['target']
sensitive_feature = df['age_group'] # Now checking fairness against Age Group

# Encode categorical variables
for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Split data (We split 'sensitive_feature' too so we can audit the Test Set)
X_train, X_test, y_train, y_test, A_train, A_test = train_test_split(
    X, y, sensitive_feature, test_size=0.2, random_state=42
)

# --- 3. TRAIN THE "BIASED" MODEL (XGBoost) ---
# We train without telling the model about fairness constraints
model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# --- 4. THE BIAS AUDIT (The "Money" Step) ---
# We use Fairlearn to compare how the model treats Older People (1) vs Younger People (0)
metric_frame = MetricFrame(
    metrics=selection_rate,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=A_test
)

print("--- INITIAL MODEL AUDIT (SENSITIVE FEATURE: AGE) ---")
print(f"Overall Accuracy: {accuracy_score(y_test, y_pred):.2%}")
print(f"Selection Rate (Overall approval %): {selection_rate(y_test, y_pred):.2%}")
print("\n--- FAIRNESS CHECK ---")
print(f"Approval Rate for Under 25 (Group 0): {metric_frame.by_group[0]:.2%}")
print(f"Approval Rate for Over 25  (Group 1): {metric_frame.by_group[1]:.2%}")
print(f"Difference: {metric_frame.difference():.2%}")

if metric_frame.difference() > 0.10:
    print("\nALERT: Significant Disparity Detected! (Gap > 10%)")
else:
    print("\nStatus: Relatively Fair")

--- INITIAL MODEL AUDIT (SENSITIVE FEATURE: AGE) ---
Overall Accuracy: 79.00%
Selection Rate (Overall approval %): 77.50%

--- FAIRNESS CHECK ---
Approval Rate for Under 25 (Group 0): 68.29%
Approval Rate for Over 25  (Group 1): 79.87%
Difference: 11.58%

ALERT: Significant Disparity Detected! (Gap > 10%)


In [17]:
from fairlearn.postprocessing import ThresholdOptimizer

# --- 5. BIAS MITIGATION (The "Fix") ---
# We wrap our biased XGBoost model in a ThresholdOptimizer.
# "demographic_parity" means we want the approval rates to be equal.
postprocess_est = ThresholdOptimizer(
    estimator=model,
    constraints="demographic_parity",
    prefit=True,
    predict_method='predict_proba'
)

# Fit the fairness wrapper on the TRAINING data (needs sensitive features)
postprocess_est.fit(X_train, y_train, sensitive_features=A_train)

# --- 6. AUDIT THE "FAIR" MODEL ---
# Now we predict using the wrapper. Note: We must provide sensitive features at prediction time.
y_pred_fair = postprocess_est.predict(X_test, sensitive_features=A_test)

# Calculate metrics for the new "Fair" model
metric_frame_fair = MetricFrame(
    metrics=selection_rate,
    y_true=y_test,
    y_pred=y_pred_fair,
    sensitive_features=A_test
)

print("\n--- MITIGATED MODEL RESULTS ---")
print(f"Overall Accuracy: {accuracy_score(y_test, y_pred_fair):.2%}")
print(f"Selection Rate: {selection_rate(y_test, y_pred_fair):.2%}")
print("\n--- FAIRNESS CHECK (AFTER FIX) ---")
print(f"Approval Rate for Under 25 (Group 0): {metric_frame_fair.by_group[0]:.2%}")
print(f"Approval Rate for Over 25  (Group 1): {metric_frame_fair.by_group[1]:.2%}")
print(f"Difference: {metric_frame_fair.difference():.2%}")

# --- 7. THE TRADEOFF REPORT ---
acc_original = accuracy_score(y_test, y_pred)
acc_fair = accuracy_score(y_test, y_pred_fair)
bias_original = metric_frame.difference()
bias_fair = metric_frame_fair.difference()

print("\n---FINAL TRADEOFF REPORT ---")
print(f"Bias Reduced By: {bias_original - bias_fair:.2%}")
print(f"Accuracy Cost:   {acc_original - acc_fair:.2%}")


--- MITIGATED MODEL RESULTS ---
Overall Accuracy: 77.50%
Selection Rate: 82.00%

--- FAIRNESS CHECK (AFTER FIX) ---
Approval Rate for Under 25 (Group 0): 90.24%
Approval Rate for Over 25  (Group 1): 79.87%
Difference: 10.37%

---FINAL TRADEOFF REPORT ---
Bias Reduced By: 1.21%
Accuracy Cost:   1.50%


C:\Users\ABCD\anaconda3\Lib\site-packages\fairlearn\postprocessing\_interpolated_thresholder.py:149: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.         1.         1.         1.         1.         1.
 1.         0.39350847 1.         1.         0.39350847 1.
 1.         1.         1.         1.         1.         1.
 0.39350847 1.         1.         0.39350847 1.         0.39350847
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         0.39350847 0.39350847 1.         1.        ]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  positive_probs[sensitive_feature_vector == a] = interpolated_predictions[


In [19]:
from fairlearn.postprocessing import ThresholdOptimizer

# --- 5. BIAS MITIGATION (ATTEMPT 2: EQUALIZED ODDS) ---
# We switch constraints to 'equalized_odds'.
# This balances the True Positive Rates (treating qualified people equally)
# rather than just forcing raw approval numbers to match.

postprocess_est = ThresholdOptimizer(
    estimator=model,
    constraints="equalized_odds",  # <--- CHANGED FROM demographic_parity
    prefit=True,
    predict_method='predict_proba'
)

# Fit on training data
postprocess_est.fit(X_train, y_train, sensitive_features=A_train)

# --- 6. AUDIT THE NEW "FAIR" MODEL ---
y_pred_fair = postprocess_est.predict(X_test, sensitive_features=A_test)

metric_frame_fair = MetricFrame(
    metrics=selection_rate,
    y_true=y_test,
    y_pred=y_pred_fair,
    sensitive_features=A_test
)

print("\n--- MITIGATED MODEL RESULTS (EQUALIZED ODDS) ---")
print(f"Overall Accuracy: {accuracy_score(y_test, y_pred_fair):.2%}")
print(f"Selection Rate: {selection_rate(y_test, y_pred_fair):.2%}")
print("\n--- FAIRNESS CHECK ---")
print(f"Approval Rate for Under 25 (Group 0): {metric_frame_fair.by_group[0]:.2%}")
print(f"Approval Rate for Over 25  (Group 1): {metric_frame_fair.by_group[1]:.2%}")
print(f"Difference: {metric_frame_fair.difference():.2%}")

# --- 7. FINAL TRADEOFF REPORT ---
acc_original = accuracy_score(y_test, y_pred)
acc_fair = accuracy_score(y_test, y_pred_fair)
bias_original = metric_frame.difference()
bias_fair = metric_frame_fair.difference()

print("\n--- FINAL TRADEOFF REPORT ---")
print(f"Original Bias Gap: {bias_original:.2%}")
print(f"New Bias Gap:      {bias_fair:.2%}")
print(f"Accuracy Cost:     {acc_original - acc_fair:.2%}")


--- MITIGATED MODEL RESULTS (EQUALIZED ODDS) ---
Overall Accuracy: 78.50%
Selection Rate: 77.00%

--- FAIRNESS CHECK ---
Approval Rate for Under 25 (Group 0): 65.85%
Approval Rate for Over 25  (Group 1): 79.87%
Difference: 14.02%

--- FINAL TRADEOFF REPORT ---
Original Bias Gap: 11.58%
New Bias Gap:      14.02%
Accuracy Cost:     0.50%


In [21]:
from fairlearn.reductions import GridSearch, DemographicParity, ErrorRate

# --- 5. THE GRID SEARCH (The "Goldilocks" Hunt) ---
# We generate 20 models, moving from "Max Accuracy" to "Max Fairness"
# This creates the "Pareto Frontier" (The Trade-off Curve)

print("--- STARTING GRID SEARCH (Training 20 Models) ---")

# 1. Define the constraint (Demographic Parity = Equal Acceptance Rates)
constraint = DemographicParity()

# 2. Setup the Grid Search
# We use a simple heavy-duty model (XGBoost) inside the grid
mitigator = GridSearch(
    estimator=xgb.XGBClassifier(objective="binary:logistic", random_state=42, n_jobs=1),
    constraints=constraint,
    grid_size=20  # Try 20 different "fairness strengths"
)

# 3. Fit (This trains 20 models, so it might take 30-60 seconds)
mitigator.fit(X_train, y_train, sensitive_features=A_train)
print("Training Complete. Analyzing Results...\n")

# --- 6. SELECT THE BEST MODEL ---
# We loop through all 20 models to find the one with:
# A) Low Bias (< 5%)
# B) High Accuracy

best_model = None
best_gap = 1.0 # Start high
best_acc = 0.0

print(f"{'ID':<5} {'Accuracy':<10} {'Bias Gap (Diff)':<20} {'Decision'}")
print("-" * 50)

for i, predictor in enumerate(mitigator.predictors_):
    # Predict with this specific candidate model
    pred_candidate = predictor.predict(X_test)
    
    # Calculate Metrics
    acc = accuracy_score(y_test, pred_candidate)
    
    # Calculate Fairness Gap (Difference between Group 0 and Group 1)
    mf = MetricFrame(metrics=selection_rate, y_true=y_test, y_pred=pred_candidate, sensitive_features=A_test)
    gap = mf.difference()
    
    # Check if this is our "Winner"
    # Criteria: Gap must be under 5% (0.05), and we want the highest accuracy possible.
    decision = ""
    if gap < 0.05 and acc > 0.73: 
        decision = "CANDIDATE"
        if acc > best_acc:
            best_model = predictor
            best_acc = acc
            best_gap = gap
            decision = "WINNER"
            
    print(f"{i:<5} {acc:.2%}     {gap:.2%}               {decision}")

# --- 7. FINAL REPORT ---
print("\n--- CHOSEN MODEL REPORT ---")
print(f"Final Accuracy: {best_acc:.2%}")
print(f"Final Bias Gap: {best_gap:.2%}")

if best_model:
    print("Use 'best_model.predict(X)' for your app!")
else:
    print("No perfect model found. You might need to relax your criteria.")

--- STARTING GRID SEARCH (Training 20 Models) ---
Training Complete. Analyzing Results...

ID    Accuracy   Bias Gap (Diff)      Decision
--------------------------------------------------
0     70.50%     84.28%               
1     68.00%     84.91%               
2     70.00%     83.65%               
3     69.50%     83.02%               
4     70.50%     83.02%               
5     70.50%     83.02%               
6     77.50%     41.48%               
7     79.50%     32.98%               
8     78.00%     23.85%               
9     79.00%     18.27%               
10    79.00%     11.58%               
11    79.50%     21.97%               
12    81.50%     17.09%               
13    79.50%     3.56%               WINNER
14    81.50%     2.58%               WINNER
15    76.50%     20.13%               
16    75.00%     24.53%               
17    77.00%     23.27%               
18    74.00%     25.79%               
19    77.00%     22.01%               

--- CHOSEN MODEL REP

In [23]:
import joblib

# 1. Grab the specific winner (Model #14 fromthe above table)
# Note: We use the index 14 because that was your winner.
final_model = mitigator.predictors_[14]

joblib.dump(final_model, 'models/fair_credit_model.pkl')

print("Model saved to models/fair_credit_model.pkl")

Model saved to models/fair_credit_model.pkl


In [25]:
import joblib
import os

save_path = '../models/fair_credit_model.pkl'

os.makedirs('../models', exist_ok=True)

try:
    final_model = mitigator.predictors_[14]
    
    joblib.dump(final_model, save_path)
    print(f"SUCCESS: Model saved to: {os.path.abspath(save_path)}")
    
except NameError:
    print("ERROR: 'mitigator' variable not found. You need to re-run the Grid Search cell first.")

SUCCESS: Model saved to: C:\Users\ABCD\models\fair_credit_model.pkl
